<a href="https://colab.research.google.com/github/ipassynk/tennis-email-agent/blob/main/tennis-email-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
!pip install crewai
!pip install crewai_tools
!pip install pydantic
!pip install -qU langchain-google-community[gmail]
!pp install google-auth-oauthlib>=1.2.0
!pp install langchain-google-community>=2.0.0
!pp install google-auth>=2.20.0
!pp install google-api-python-client>=2.100.0

/bin/bash: line 1: pp: command not found
/bin/bash: line 1: pp: command not found
/bin/bash: line 1: pp: command not found
/bin/bash: line 1: pp: command not found


In [113]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool
from crewai_tools import ScrapeElementFromWebsiteTool
from langchain_google_community import GmailToolkit
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, List
import os
import json

In [114]:
website_url = 'https://torontowinterleague.tenniscores.com/?mod=nndz-TjJiOWtORzkwTlJFb0NVU1NzOD0%3D&team=nndz-WVNXOHhMOD0%3D'
club_name = "Thornhill Park Tennis Club"
date = "11/09"

In [115]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [116]:
scrape_tool = ScrapeElementFromWebsiteTool(website_url=website_url, css_element=".team_schedule")

scraper_agent = Agent(
    role="Web Scraper",
    goal="Extract specific information from websites",
    backstory="An expert in web scraping who can extract targeted content from web pages.",
    tools=[scrape_tool],
    verbose=True,
    llm="gpt-4o"
)

scrape_task = Task(
    description="Extract row from a schedule table. Use the CSS selector '.team_schedule' to target the table and find row with date = '12/07'.",
    expected_output="A list of the main headlines from CNN.",
    agent=scraper_agent,
)

In [117]:
class GetCapitanInput(BaseModel):
    team: str = Field(..., description="Name of the team to lookup")

class GetCapitan(BaseTool):
    name: str = "get_capitan"
    description: str = "Fetch capitans emails and names by team name from teams.json"
    args_schema: Type[BaseModel] = GetCapitanInput

    def _run(self, team: str) -> List[dict]:
        try:
            with open("teams.json", "r") as f:
                data = json.load(f)
                capitans = [entry for entry in data if entry['team'].lower() == team.lower()]
                if not capitans:
                    return [{"message": f"No capitans for for team {team}"}]
                return capitans
        except Exception as e:
            return [{"error": str(e)}]

capitan_tool = GetCapitan()

capitan_agent = Agent(
    role="Lookup email and name by team name",
    goal="Find email and name by team name",
    backstory=(
        "You can find opponet capitain email and name by team name. "
    ),
    tools=[capitan_tool],
    verbose=True,
    llm="gpt-4o"
)

capitan_task = Task(
    description="Find capitains emails and name for a team by name",
    expected_output="A list of dictionaries containing the name and email of the team's capitain(s).",
    agent=capitan_agent,
)

In [118]:
class GetMatchEmailInput(BaseModel):
    date: str = Field(..., description="Match date. For example: '2025-10-27'")
    time: str = Field(..., description="Match time. For example: '7:00pm'")

class GetMatchEmail(BaseTool):
    name:str = "get_match_email"
    description: str = "Create an email subject and body for a match at Thornhill Park Tennis Club. Input: match_date (string), optional time."
    args_schema: Type[BaseModel] = GetMatchEmailInput

    def _run(self, date: str, time: str):
        subject = f"Match {date} {time} at {club_name}"

        body = f"""Hi Ladies,

            Our teams are scheduled to play on {date} at {time} at the {club_name}.
            The club address is 26A Old Yonge St, Thornhill, ON L4J 8C5 (at the corner of Centre St and Yonge St).
            There’s a large parking lot near the clubhouse, but you’ll need to obtain temporary parking permits. Please pick them up from the monitor inside the clubhouse and display them on your dashboards. Additional parking is available just across Old Yonge St near the Thornhill Pub.
            Looking forward to seeing your team tonight!

            Cheers,
            Julia Passynkova
        """
        return {"body": body, "subject": subject}

email_tool = GetMatchEmail()

email_agent = Agent(
    role="Tennis email writter",
    goal="Create draft email",
    backstory=(
        "You can create draft email for oppoonet team having subject and body. "
    ),
    tools=[email_tool],
    verbose=True,
    llm="gpt-4o"
)

email_task = Task(
    description="Create draft reminder email for match.",
    expected_output="Create email body and subject.",
    agent=email_agent,
)

In [119]:
from langchain_google_community import GmailToolkit
from crewai.tools import BaseTool # Import BaseTool
import pickle
from googleapiclient.discovery import build

def get_gmail_service():
    """Get Gmail service with OAuth credentials."""
    creds = None
    # The file token.pickle stores the user's access and refresh tokens.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    service = build('gmail', 'v1', credentials=creds)
    return service

service = get_gmail_service()
gmail_toolkit = GmailToolkit(api_resource=service)
langchain_gmail_tools = gmail_toolkit.get_tools()

In [120]:
from langchain_google_community import GmailToolkit
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

service = get_gmail_service()
toolkit = GmailToolkit(api_resource=service)
langchain_gmail_tools = gmail_toolkit.get_tools()

class GmailCreateDraftTool(BaseTool):
    name: str = "gmail_create_draft"
    description: str = "Creates a draft email in Gmail."

    def __init__(self, langchain_tool):
        super().__init__()
        self.langchain_tool = langchain_tool
        self.name = langchain_tool.name
        self.description = langchain_tool.description
        # Assuming LangChain tool has an args_schema
        if hasattr(langchain_tool, 'args_schema'):
            self.args_schema = langchain_tool.args_schema


    def _run(self, *args, **kwargs):
        return self.langchain_tool.run(*args, **kwargs)


tool_wrappers = {
    "gmail_create_draft": GmailCreateDraftTool,
}

# Create CrewAI compatible tools using the wrapper classes
crewai_gmail_tools = []
for tool in langchain_gmail_tools:
    wrapper_class = tool_wrappers.get(tool.name)
    if wrapper_class:
        crewai_gmail_tools.append(wrapper_class(tool))


gmail_agent = Agent(
    tools=crewai_gmail_tools, # Use the wrapped tools
    role="Tennis email sender",
    goal="Send draft email via GMail",
    backstory=(
        "You can send draft email for oppoonet emails so I can review and forward the email myself. "
    ),
    verbose=True,
    llm="gpt-4o"
)

gmail_task = Task(
    description="Draft and send a reminder email for match.",
    expected_output="A sent email confirming match details.",
    agent=gmail_agent,
)

In [121]:
supervisor_agent = Agent(
    role="Supervisor Agent",
    goal="Coordinate tasks among team lookup and email agents to prepare match communications.",
    backstory="Understands how to use other agents to accomplish composite goals.",
    tools=[scrape_tool, capitan_tool, email_tool] + crewai_gmail_tools,
    reasoning=True,
    max_reasoning_attempts=3,
    verbose=True,
    llm="gpt-4o"
)

supervisor_task = Task(
    description=(
        "Plan and coordinate the process: "
        "Find if Thornhill Park team plays at home (H) for a specific date. "
        "If Thornhill Park team plays away game (A) tell this and stop the flow. "
        "Get opponent team name. "
        "Find opponent email and name. "
        "Create email body and subject. "
        "Send draft email for opponent emails so I can review and forward the email myself. "
    ),
    expected_output="Confirmation of whether the Thornhill Park team plays at home or away on the specified date, and if playing at home, the drafted email details for the opponent team's captain(s).",
    agent=supervisor_agent,
    dependencies=[scrape_task, capitan_task, email_task, gmail_task]
)

In [122]:
crew = Crew(
  agents=[supervisor_agent],
  tasks=[supervisor_task],
  verbose=True,
  process=Process.sequential,
  color=False
  #inject_date=True,  # Automatically inject current date into tasks
  #date_format="%B %d, %Y",  # Format as "May 21, 2025"
)

In [123]:
inputs = {
    "date": date,
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c888ddd6-b6b2-420c-85cc-0f9fa30d1172                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🧠 Reasoning Plan ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Here is my detailed plan for accomplishing the task at hand:                                                   │
│                                                                                                                 │
│  1. **Understanding the Task**:                                                                                 │
│     - The primary goal is to determine the location of a match for the Thornhill Park team and, if it's a home  │
│  game, to draft an email for the opponent team's captain(s) for review and sending.                             │
│     - Key components include verifying the match location, identifying the opponent team, finding relevant      │
│  contact information, and crafting a proper email communication.                                                │
│                                                                                                                 │
│  2. **Key Steps**:                                                                                              │
│     - **Step 1**: Verify if the Thornhill Park team plays at home or away on a specific date.                   │
│       - Use tool 1: Read a website content for schedule details.                                                │
│     - **Step 2**: If the team plays at home, identify the opponent team.                                        │
│     - **Step 3**: Retrieve the email and name of the opponent team's captain.                                   │
│       - Use tool 2: `get_capitan` for fetching the opponent captain’s contact details.                          │
│     - **Step 4**: Draft the email subject and body.                                                             │
│     - **Step 5**: Prepare a draft email consisting of the collected information for review.                     │
│       - Use tool 3: `get_match_email` for drafting the email with placeholders for attributes like the          │
│  opponent team name and date.                                                                                   │
│                                                                                                                 │
│  3. **Approach to Challenges**:                                                                                 │
│     - **Challenge 1**: Ensuring accurate match information – rely on the accuracy and updated nature of         │
│  website content to confirm if it's a home or away game.                                                        │
│     - **Challenge 2**: Correct identification of opponent contact details – ensure the functions used are       │
│  up-to-date and querying the correct database for the most accurate email and name.                             │
│     - **Challenge 3**: Efficient communication – clear email template that requires minimal editing for         │
│  approval and final sending.                                                                                    │
│                                                                                                                 │
│  4. **Strategic Use of Tools**:                                                                                 │
│     - **Tool Usage**:                                                                                           │
│       - Utilize “Read a website content” to determine if Thornhill Park plays at home. This is the initial and  │
│  most crucial step for branching the workflow.                                                                  │
│       - If a home game is confirmed, apply `get_capitan` to fetch the captain’s name and email for the away     │
│  team.                                                 

Output()

Repaired JSON: {}


[CrewAIEventsBus] Sync handler error in on_agent_logs_execution: Extra data: line 2 column 1 (char 3)

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Supervisor Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  On 11/30, the Thornhill Park team plays an away game against Veneto A2, so no further actions are needed as    │
│  the match is not at home.                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯